In [80]:
#调用numpy拓展包
import numpy

#调用scipy.special
import scipy.special

#调用绘图包
import matplotlib.pyplot
%matplotlib inline

In [81]:
# 神经网络类

class neuralnetwork:
    
    # 神经网络类初始化
    def __init__(self,innode,hidenode,outnode,learningrate): 
        
        # 输入层节点  隐藏节点 输出层节点 学习率 
        self.innodes=innode
        self.hidenodes=hidenode
        self.outnodes=outnode
        self.study=learningrate
        
        #构建初始权重矩阵
            #numpy. random. rand( rows , columns ) 生成随机矩阵 且矩阵的值为0-1
            #-0.5 权重出现正负 整体权重在-0.5和0.5之间波动
        #self.in_hid=(numpy.random.rand(self.hidenodes, self.innodes)-0.5)
        #self.hid_out=(numpy.random.rand(self.outnodes, self.hidenodes)-0.5)
        
        #进行优化之后的初始权重矩阵
            #numpy.random.normal() 以正态分布的方式采样
        self.in_hid=numpy.random.normal(0.0,pow(self.hidenodes,-0.5),(self.hidenodes, self.innodes))
        self.hid_out=numpy.random.normal(0.0,pow(self.outnodes,-0.5),(self.outnodes, self.hidenodes))
        
        #激活函数
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    # 模型训练
    def train(self,input_list,target_list):
        
        #数据传输
        inputs=numpy.array(input_list,ndmin=2).T
        targets=numpy.array(target_list,ndmin=2).T
        
        #计算隐藏层输入信号
        hide_in= numpy.dot(self.in_hid,inputs)
        
        #计算隐藏层输出信号
        hide_out=self.activation_function(hide_in)
        
        #计算最后的输入信号
        final_in=numpy.dot(self.hid_out, hide_out)
        
        #计算最后的输出信号
        final_out=self.activation_function(final_in)
        
        #输出层误差值计算
        out_error=targets-final_out
        
        #隐藏层误差值计算
        hide_error=numpy.dot(self.hid_out.T, out_error)
        
        #更新权重1
        self.hid_out+=self.study* numpy.dot((out_error*final_out*(1.0-final_out)),numpy.transpose(hide_out))
        
        #更新权重2
        self.in_hid+=self.study*numpy.dot((hide_error*hide_out*(1.0-hide_out)),numpy.transpose(inputs))
        
        pass

    # 问答
    def query(self,input_list): 
        
        #传递输入数据列表
        inputs=numpy.array(input_list,ndmin=2).T
        
        #计算隐藏层的输入信号
        hide_in=numpy.dot(self.in_hid,inputs)
        
        #计算隐藏层输出信号
        hide_out=self.activation_function(hide_in)
        
        #计算最终的输入信号
        final_in= numpy.dot(self.hid_out, hide_out)
        
        #计算最终的输出信号
        final_out=self.activation_function(final_in)
        
        return final_out

In [82]:
# 赋值构建神经网络对象

tmp_in=784
tmp_hide=200
tmp_out=10
tmp_lr=0.1

tmp=neuralnetwork(tmp_in,tmp_hide,tmp_out,tmp_lr)

In [83]:
#加载训练文件

training_file=open("/Users/dinghao/Documents/手写数字识别/mnist_train.csv", 'r')
training_list=training_file.readlines()
training_file.close()

In [84]:
# 神经网络训练代码

for i in range(5):
    
    #遍历所有训练样本
    for record in training_list:
        
        all_values=record.split(',')
        inputs=(numpy.asfarray(all_values[1:])/255.0*0.99)+0.01
        targets=numpy.zeros(tmp_out)+0.01
        targets[int(all_values[0])]=0.99
        tmp.train(inputs,targets)
        pass
    
    pass

In [85]:
#加载神经网络测试数据

test_file=open("/Users/dinghao/Documents/手写数字识别/mnist_test.csv", 'r')
test_list=test_file.readlines()
test_file.close()

In [86]:
#神经网络测试

scorecard = []

for record in test_list:
    
    all_values=record.split(',')
    
    #第一个数据为正确答案
    ans=int(all_values[0])
    
    inputs=(numpy.asfarray(all_values[1:])/255.0*0.99)+0.01
    outputs=tmp.query(inputs)
    label=numpy.argmax(outputs)
    
    #结果检验
    if (label==ans):
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass
    pass

In [87]:
#计算神经网络的正确率并输出相应的结果

scorecard_array=numpy.asarray(scorecard)
print ("performance = ",scorecard_array.sum()/scorecard_array.size)

performance =  0.9739
